# DEEP AUTOENCODER WITH KERAS

Autoencoders are a popular class of Deep Neural Networks that are capable of learning representations in a self-supervised way. We will build one with Keras.

**a) Fetch the MNIST dataset ([https://www.tensorflow.org/api\_docs/python/tf/keras/datasets/mnist/load\_data](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data)) with Keras and scale the data to \[0,1].**

**b) Build a convolutional autoencoder model consisting of encoder and decoder.**

*   The encoder and decoder are structured in symmetrical fashion, i.e., both parts use the same layer structure.
*   The encoder uses two convolutional layers (64 and 32 filters) followed by a flattening layer and a dense layer. The number of neurons of this dense layer defines the encoded vector size of the input image. Set it to 64. Use parameters of your choice for all other layers.
*   The decoder transforms the input data, i.e., the encoded vector coming from the encoder, in reverse order compared to the encoder. An upscaling dense layer followed by two deconvolutional (transposed) layers and a final convolutional layer with a single filter, all layers using similar parameters as the encoder layers.
*   Explain the purpose of the individual layers and how the input shapes change during the transformations of these layers.

**c) Train this model using a binary cross-entropy loss for three epochs. Use the SGD optimizer with a learning rate of 1.5.**

**d) Select five random images from the test set and compare the visualizations of the original and the auto-encoded images.**

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import numpy as np

In [8]:
# a)
# Load the MNIST dataset
data = mnist.load_data()
X = data[0][0]
y = data[0][1]
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Normalize the data to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape data to include channel dimension
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")


Train shape: (48000, 28, 28, 1), Test shape: (12000, 28, 28, 1)


In [6]:
# b)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Conv2DTranspose, Reshape

# Encoder
input_img = Input(shape=(28, 28, 1))

# Encoder layers
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
encoded = Dense(64, activation='relu')(x)

# Decoder layers
x = Dense(7 * 7 * 32, activation='relu')(encoded)
x = Reshape((7, 7, 32))(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='SGD', loss='binary_crossentropy')

autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1568)              0     

In [7]:
# c)
from tensorflow.keras.optimizers import SGD

# Set the optimizer with a learning rate of 1.5
optimizer = SGD(learning_rate=1.5)
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')

# Train the model
autoencoder.fit(x_train, x_train, epochs=3, batch_size=256, validation_data=(x_test, x_test))

Epoch 1/3
188/188 [==============================] - ETA: 0s - loss: 0.3870WARNING:tensorflow:Model was constructed with shape (None, 28, 28, 1) for input KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shape (None, 1).


ValueError: in user code:

    File "d:\Data science\Python materials\Machine learning\Machine-learning\.venv\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "d:\Data science\Python materials\Machine learning\Machine-learning\.venv\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Data science\Python materials\Machine learning\Machine-learning\.venv\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "d:\Data science\Python materials\Machine learning\Machine-learning\.venv\lib\site-packages\keras\engine\training.py", line 1499, in test_step
        y_pred = self(x, training=False)
    File "d:\Data science\Python materials\Machine learning\Machine-learning\.venv\lib\site-packages\keras\utils\traceback_utils.py", line 67, in err

---


# DENOISING AUTOENCODER WITH KERAS

In this exercise, we will use autoencoders for the purpose of denoising data. The approach will be similar to that of Exercise 1.

**a) Load the MNIST dataset and scale it to \[0,1].**

**b) Build an autoencoder model of your choice. The encoder must produce vectorial representations of size 64. You can use a convolution model as in Exercise 1 or use a simple model of dense layers.**

**c) Add three different forms of noise to the training and test data (one noisy dataset for each type of noise). Make sure that the resulting images are still in the range \[0,1]. Also, use a method that allows a factor to control the amount of noise that is added. Use 0.25 as the value for tests:**

1.  Gaussian noise with a mean of 0.0 and a standard deviation of 1.0.
2.  Uniform noise from a range of \[0,1].
3.  Noise in the form of merged images. Apply noise to one image by merging it with another random image from the examples.

**d) Train your model for three epochs using a suitable loss function.**

*   Use a combined dataset with all noisy examples as input data and use the original examples as target data.
*   Use the SGD optimizer with a learning rate of 1.5.

**e) After training, visualize the model’s performance on a single digit. Display the original image, all three noisy variants, and the denoised image.**


------


# SUPERVISED REPRESENTATION LEARNING

In contrast to the exercises from before, learning representations can also be done supervised. This is especially useful in scenarios where labels for similar data instances exist that can guide the learning process.

**a) Load the MNIST dataset and scale the data to the range \[0,1].**

**b) Prepare a training data generator ([https://www.tensorflow.org/api\_docs/python/tf/data/Dataset#from\_generator](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_generator)). Each training instance is put together from two images and a label that is either 1 or 0. Randomly, generate training instances by putting together two images of the same class (with label 1) or two random images (with label 0).**

**c) Build a model using the Functional API of TensorFlow ([https://www.tensorflow.org/guide/keras/functional](https://www.tensorflow.org/guide/keras/functional)).** 

*   The model should have a sub-model that transforms a single image into a representation of size 64. 
*   The main model should take the pairs of images and transform both of them to their vector representation using the same sub-model. Afterward, a pairwise dot product should be computed for the vector pair. This dot product is the final output.

**d) Train the model with the priorly created generator for five epochs, using hinge loss.**

*   Why is this loss function used?
*   How does it work?


---


# REINFORCEMENT LEARNING FROM SCRATCH

In this exercise, we will build up a Reinforcement Learning (RL) process. We will train a model to help us play Tic Tac Toe. Use the provided class `tic_tac_toe.py` for this exercise.

**a) Get familiar with the provided class `TicTacToe` by creating a game and playing a few rounds with random choices. The AI player is supposed to be the player 'o'.**

**b) Collect the information of 1000 games that can be used to train a model later on:**

*   Set up a loop that plays 1000 games.
*   Each game is played by both players alternating. Player 'x' (You) always begins with a random play. Afterward, player 'o' makes its random move.
*   For each of your turns, i.e., the turns of player 'x', store the board state before the turn and the cell that was chosen (information is returned by the random play method and can be retrieved from the board object).
*   Always check if the game is finished after a player’s turn and if so, compute the rewards for this game (use the provided function `get_rewards()` in the file). This method expects the winner of the game as a parameter and a list of player x’s plays as tuples of chosen cell and board state before play.

**c) The rewards are given by this function as a tuple where the first value is the rewards (target) and the second value is the training data in the form of chosen cell and board state.**

*   Train a model of your choice (can be a Deep Learning Model or another model) to predict the expected reward given the provided training data.
*   Split the available data before training (test ratio of 0.2).

**d) Evaluate the trained model on the test set in terms of prediction deviation. Explain how the trained model could be used to help human players win Tic Tac Toe games.**
